In [1]:
!pip install gym
!pip install stable-baselines[mpi]
!pip install tensorflow==1.14.0

  Using cached gym-0.18.0.tar.gz (1.6 MB)
  Using cached pyglet-1.5.0-py2.py3-none-any.whl (1.0 MB)
  Using cached cloudpickle-1.6.0-py3-none-any.whl (23 kB)
  Using cached future-0.18.2.tar.gz (829 kB)
  Created wheel for gym: filename=gym-0.18.0-py3-none-any.whl size=1656449 sha256=a6fc9cd1f70724551060ee228b474e1af7bda02ae04e871d3d051ed80703ef5d
  Stored in directory: c:\users\61431\appdata\local\pip\cache\wheels\dd\c7\f0\a102782c25a396951685b12821797eaaad88e802e50271916f
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491059 sha256=df176b02c6506cadff1f1314a37a67cfecb09e10f59acbb623e76405037a4203
  Stored in directory: c:\users\61431\appdata\local\pip\cache\wheels\6e\9c\ed\4499c9865ac1002697793e0ae05ba6be33553d098f3347fb94
Successfully built gym future
  Using cached stable_baselines-2.10.1-py3-none-any.whl (240 kB)
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)
  Using cached termcolor-1.1.0.tar.gz (3.9 kB)
  Using cached astor-0.8.1-py2.py3-none

Environment

In [3]:
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import PPO2

C:\Users\61431\anaconda3\envs\RL New\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\61431\anaconda3\envs\RL New\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\61431\anaconda3\envs\RL New\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\61431\anaconda3\envs\RL New\lib\site-packages\tensorflow\pyth

In [19]:
import gym
from gym import spaces

class ATCEnv(gym.Env):
    metadata = {'render.modes': ['human']}
    
    def __init__(self, noOfPlanes, noOfRunways, planeToTypeMapping, runwayToTypeMapping, planeToStatusMapping):
        super(ATCEnv, self).__init__()
        
        self.planeType = ['small','mid', 'jumbo']
        self.runwayType = ['short', 'long', 'emergency']
        self.status = ['normal', 'emergency']
        
        self.noOfPlanes = noOfPlanes
        self.noOfRunways = noOfRunways
       
        self.totalNoOfPlanes = sum(noOfPlanes)
        self.totalNoOfRunways = sum(noOfRunways)
        
        
       
        self.planeToTypeMapping = planeToTypeMapping
        self.runwayToTypeMapping = runwayToTypeMapping
        self.planeToStatusMapping = planeToStatusMapping
     
        
        self.action_space = spaces.Tuple(
                (spaces.Discrete(len(self.noOfRunways)), 
                (spaces.Discrete(len(self.noOfPlanes)))))
                 
        #self.observation_space = {}
        
        self.observation_space = {}
        self.obs = {}
        self.act = {}

            
        self.reward = 0
        self.current_step = 0
    
    def reset(self):
            self.reward = 0
            self.action_space = spaces.Tuple(
                (spaces.Discrete(len(self.noOfRunways)), 
                (spaces.Discrete(len(self.noOfPlanes)))))
            
            self.observation_space ={}
        
       
            self.current_step = 0
            self.obs = {}
            self.act = {}
            
            #self.noOfRunways = noOfRunways
          
            return self.observation_space
            
    def step(self, action):
        runway = action[0]
        plane = action[1]
        
        r = self.reward
        
        print(plane)
        
        if self.runwayToTypeMapping[runway] == self.runwayType[2] and self.planeToStatusMapping[plane] == self.status[1]:
            self.reward = self.reward + 1
        elif self.planeToStatusMapping[plane] != self.status[1] and self.runwayToTypeMapping[runway] not in ['short','emergency'] and self.planeToTypeMapping[plane] in ['mid', 'jumbo']:
            self.reward = self.reward + 1
        elif self.planeToStatusMapping[plane] != self.status[1] and self.runwayToTypeMapping[runway] == 'short' and self.planeToTypeMapping[plane] == 'small':
            self.reward = self.reward + 1
        else:
            self.reward = self.reward - 1

        self.current_step = self.current_step + 1
        
        if r <= self.reward:
            self.observation_space[runway] = plane
            obs = action
            self.obs = obs
        else:
            obs = self.obs
        act = action
        
        
        #print(self.noOfRunways)
        #print('runway',runway)
        #self.noOfRunways.remove(runway)
        
        #print(self.noOfRunways)
        
        
        
        done = len(self.observation_space) == len(self.noOfRunways)
        
        print('o',self.observation_space)
        print('r',self.noOfRunways)
        print('reward', self.reward)
        return act, obs, done

        
              

In [20]:
import random

noOfPlanes = [0,1,2,3]
noOfRunways = [0,1,2,3] 
planeToTypeMapping = {0:'small',1:'small',2:'mid',3:'jumbo'}
runwayToTypeMapping = {0:'emergency',1:'short',2:'long',3:'long'}
planeToStatusMapping = {0:'normal',1:'normal',2:'normal',3:'emergency'}


env = ATCEnv(noOfPlanes,noOfRunways,planeToTypeMapping,runwayToTypeMapping,planeToStatusMapping)
#env = DummyVecEnv([lambda: ATCEnv(noOfPlanes,noOfRunways,planeToTypeMapping,runwayToTypeMapping,planeToStatusMapping)])

#model = PPO2(MlpPolicy, env, verbose=1)
#model.learn(total_timesteps=100)
obs = env.reset()

#obs = env.reset()
print(obs)

for t in range(100):
        #env.render()
        print(obs)
        
       # key = random.choice(list(env.action_space.keys()))
       # val = env.action_space[key]
        action = env.action_space.sample()
        #action, _states = model.predict(obs)
        obs, reward, done = env.step(action)
        print(obs, reward, done)
        if done:
            print("Finished after {} timesteps".format(t+1))
            break

{}
{}
1
o {}
r [0, 1, 2, 3]
reward -1
(2, 1) {} False
(2, 1)
3
o {0: 3}
r [0, 1, 2, 3]
reward 0
(0, 3) (0, 3) False
(0, 3)
0
o {0: 3, 1: 0}
r [0, 1, 2, 3]
reward 1
(1, 0) (1, 0) False
(1, 0)
3
o {0: 3, 1: 0}
r [0, 1, 2, 3]
reward 0
(3, 3) (1, 0) False
(3, 3)
3
o {0: 3, 1: 0}
r [0, 1, 2, 3]
reward -1
(1, 3) (1, 0) False
(1, 3)
3
o {0: 3, 1: 0}
r [0, 1, 2, 3]
reward -2
(2, 3) (1, 0) False
(2, 3)
1
o {0: 3, 1: 0}
r [0, 1, 2, 3]
reward -3
(0, 1) (1, 0) False
(0, 1)
1
o {0: 3, 1: 0}
r [0, 1, 2, 3]
reward -4
(3, 1) (1, 0) False
(3, 1)
0
o {0: 3, 1: 0}
r [0, 1, 2, 3]
reward -5
(0, 0) (1, 0) False
(0, 0)
3
o {0: 3, 1: 0}
r [0, 1, 2, 3]
reward -4
(0, 3) (0, 3) False
(0, 3)
1
o {0: 3, 1: 0}
r [0, 1, 2, 3]
reward -5
(2, 1) (0, 3) False
(2, 1)
0
o {0: 3, 1: 0}
r [0, 1, 2, 3]
reward -6
(2, 0) (0, 3) False
(2, 0)
0
o {0: 3, 1: 0}
r [0, 1, 2, 3]
reward -5
(1, 0) (1, 0) False
(1, 0)
3
o {0: 3, 1: 0}
r [0, 1, 2, 3]
reward -6
(3, 3) (1, 0) False
(3, 3)
0
o {0: 3, 1: 0}
r [0, 1, 2, 3]
reward -7
(3, 0) (1